In [6]:
from PIL import Image
import json
import numpy as np                      
from skimage import measure                        
from shapely.geometry import Polygon, MultiPolygon 


def create_sub_masks(mask_image):
    #width, height = mask_image.size
    width = mask_image.width
    height = mask_image.height
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(0, width):
        for y in range(0, height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))
            #pixel = str(r,g,b)

            # If the pixel is not black...
            if (pixel) != (0, 0, 0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [7]:
def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [8]:
plant_book_mask_image = Image.open('colored_label_image.png').convert('RGB')
bottle_book_mask_image = Image.open('colored_label_image2.png').convert('RGB')

mask_images = [plant_book_mask_image, bottle_book_mask_image]

# Define which colors match which categories in the images
circle, background, rectangle1, rectangle2 = [1, 2, 3, 4]
category_ids = {
    1: {
        '(255, 255, 0)': circle,
        '(0, 0, 255)': background,
        '(255, 0, 0)': rectangle1
    },
    2: {
        '(255, 255, 0)': rectangle1,
        '(0, 0, 255)': circle,
        '(255, 0, 255)': rectangle2,
        '(255, 0, 0)': background
    }
}

is_crowd = 0

# These ids will be automatically increased as we go
annotation_id = 1
image_id = 1

# Create the annotations
annotations = []
for mask_image in mask_images:
    sub_masks = create_sub_masks(mask_image)
    for color, sub_mask in sub_masks.items():
        category_id = category_ids[image_id][color]
        annotation = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
        annotations.append(annotation)
        annotation_id += 1
    image_id += 1

print(json.dumps(annotations))

[{"segmentation": [[1.0, 91.5, 4.5, 88.0, 5.0, 83.5, 13.0, 82.5, 17.0, 77.5, 26.0, 72.5, 71.0, 31.5, 75.0, 30.5, 100.0, 5.5, 104.0, 4.5, 107.5, 0.0, 14.0, -0.5, 12.5, 8.0, 9.0, 11.5, 0.0, 10.5, -0.5, 91.0, 1.0, 91.5]], "iscrowd": 0, "image_id": 1, "category_id": 3, "id": 1, "bbox": [-0.5, -0.5, 108.0, 92.0], "area": 4954.0}, {"segmentation": [[311.0, 277.5, 317.5, 273.0, 318.5, 14.0, 315.5, 8.0, 315.0, -0.5, 131.0, -0.5, 130.5, 1.0, 138.5, 12.0, 137.5, 24.0, 130.0, 31.5, 124.0, 33.5, 118.0, 33.5, 109.0, 30.5, 104.0, 35.5, 100.0, 36.5, 81.0, 56.5, 73.0, 63.5, 69.0, 64.5, 55.5, 78.0, 55.5, 81.0, 59.5, 85.0, 59.5, 94.0, 52.5, 99.0, 52.5, 101.0, 57.0, 103.5, 67.0, 103.5, 69.0, 105.5, 74.0, 106.5, 77.5, 110.0, 81.5, 120.0, 88.5, 128.0, 88.5, 139.0, 80.0, 147.5, 71.0, 150.5, 66.0, 150.5, 62.0, 152.5, 51.0, 151.5, 38.5, 139.0, 38.5, 124.0, 41.5, 121.0, 42.5, 113.0, 48.5, 108.0, 46.0, 105.5, 42.0, 105.5, 31.0, 115.5, 27.0, 116.5, 22.5, 121.0, 21.5, 125.0, 10.5, 134.0, 10.5, 166.0, 6.0, 166.5, 